In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from statistics import median, mean
import numpy as np
from coordinate_converter import utm_to_ssb_grid_id

print("loading data")
df = pd.read_csv("proprietary_data/incidents_processed_3.csv", encoding='utf-8', escapechar='\\', parse_dates=True, low_memory=False)
urgencies = [["A"], ["H"], ["V", "V1", "V2"]]

In [ ]:
start_datetime = pd.to_datetime("2017-08-07 00:00:00")  # Specify the start datetime
end_datetime = pd.to_datetime("2017-08-14 00:00:00")  # Specify the end datetime

df = df[(pd.to_datetime(df["tidspunkt"]) >= start_datetime) & (pd.to_datetime(df["tidspunkt"]) <= end_datetime)]

In [ ]:
print("calculating delay")
df["delay"] = pd.to_datetime(df["rykker_ut"]) - pd.to_datetime(df["varslet"])

stop_point = -1
time_max = 60*15
time_min = 60

medians = []

print("filter and sort:")
for urgency in tqdm(urgencies):
    df_urgency = df[df["hastegrad"].isin(urgency)]
    df_urgency = pd.TimedeltaIndex(data = df_urgency["delay"])
    df_urgency = df_urgency.seconds.values
    df_urgency = [u for u in df_urgency if time_min < u < time_max]
    df_urgency.sort()
    df_urgency = df_urgency[:stop_point]
    medians.append(median(df_urgency))
    plt.plot(range(len(df_urgency)), df_urgency, label=urgency[0])

print(medians)

plt.legend()
plt.show()

In [ ]:
print("calculating delay")
df["handling"] = pd.to_datetime(df["varslet"]) - pd.to_datetime(df["tidspunkt"])

stop_point = -1
time_max = 60*10 
time_min = 0

medians_2 =[]

print("filter and sort:")
for urgency in tqdm(urgencies):
    df_urgency = df[df["hastegrad"].isin(urgency)]
    df_urgency = pd.TimedeltaIndex(data = df_urgency["handling"])
    df_urgency = df_urgency.seconds.values
    df_urgency = [u for u in df_urgency if time_min < u < time_max]
    df_urgency.sort()
    df_urgency = df_urgency[:stop_point]
    medians_2.append(median(df_urgency))
    plt.plot(range(len(df_urgency)), df_urgency, label=urgency[0])

print(medians_2)

plt.legend()
plt.show()

In [ ]:
od_paths = {}
with open(f'data/od_paths.json', 'r') as r:
    od_paths = json.load(r)

In [ ]:
import numpy as np

print("calculating delay")
df["hospital"] = pd.to_datetime(df["ledig_transport"]) - pd.to_datetime(df["ank_levsted"])

stop_point = -1
time_max = 60*6000
time_min = 0

medians_3 = []

print("filter and sort:")
for urgency in tqdm(urgencies):
    df_urgency = df[df["hastegrad"].isin(urgency)]
    df_urgency = pd.TimedeltaIndex(data = df_urgency["hospital"])
    df_urgency = df_urgency.seconds.values
    df_urgency = [u for u in df_urgency if time_min < u < time_max]
    df_urgency.sort()
    df_urgency = df_urgency[:stop_point]
    medians_3.append(median(df_urgency))
    plt.plot(range(len(df_urgency)), df_urgency, label=urgency[0])

print(medians_3)

plt.legend()
plt.show()